# Logging

# Sources

- <a href="https://habr.com/ru/companies/wunderfund/articles/683880/">Logging in python: developer guide (rus)</a>;

In [1]:
import os
import sys
import logging
import random

# **Basics**

## Error types

There are five levels of logging:

- Debug;
- Info;
- Warning;
- Error;
- Critical.

You can call them by using the relevant functions:

In [2]:
logging.debug("A DEBUG Message")
logging.info("An INFO")
logging.warning("A WARNING")
logging.error("An ERROR")
logging.critical("A message of CRITICAL severity")

ERROR:root:An ERROR
CRITICAL:root:A message of CRITICAL severity


In previous examle `debug` and `info` errors wasn't printed. It happened becaulse default value for `logging.level` setted to `logging.WARNING`, therefore only those messages above `WARNING` will be displayed. 

## `basicConfig`

In [2]:
logging.basicConfig(level=logging.INFO)

logging.debug("A DEBUG Message")
logging.info("An INFO")
logging.warning("A WARNING")
logging.error("An ERROR")
logging.critical("A message of CRITICAL severity")

INFO:root:An INFO
ERROR:root:An ERROR
CRITICAL:root:A message of CRITICAL severity


## `getLogger` - creating loggers

For some reason you can't set basic config twice with `logging` lib. I showing it in the following example:

- Set `level=logging.DEBUG` and, as expected, got every type of message;
- Set `level=logging.ERROR` and, but any way got every type of message.

In [6]:
print("=====logging run1=====", file=sys.stderr)
logging.basicConfig(level=logging.DEBUG)
logging.debug("A DEBUG Message")
logging.info("An INFO")
logging.warning("A WARNING")
logging.error("An ERROR")
logging.critical("A message of CRITICAL severity")

print("=====logging run2=====", file=sys.stderr)
logging.basicConfig(level=logging.ERROR)
logging.debug("A DEBUG Message")
logging.info("An INFO")
logging.warning("A WARNING")
logging.error("An ERROR")
logging.critical("A message of CRITICAL severity")

=====logging run1=====
DEBUG:root:A DEBUG Message
INFO:root:An INFO
ERROR:root:An ERROR
CRITICAL:root:A message of CRITICAL severity
=====logging run2=====
DEBUG:root:A DEBUG Message
INFO:root:An INFO
ERROR:root:An ERROR
CRITICAL:root:A message of CRITICAL severity


The solution for different situations is to create different loggers with different options. In the following example, I create different loggers with different `level` options, so that `logger2` successfully sets the `ERROR` option.

In [11]:
logger1 = logging.getLogger("logger1")
logger1.setLevel(logging.INFO)

print("=====logger1=====", file=sys.stderr)
logger1.debug("A DEBUG Message")
logger1.info("An INFO")
logger1.warning("A WARNING")
logger1.error("An ERROR")
logger1.critical("A message of CRITICAL severity")


logger2 = logging.getLogger("logger2")
logger2.setLevel(logging.ERROR)

print("=====logger2=====", file=sys.stderr)
logger2.debug("A DEBUG Message")
logger2.info("An INFO")
logger2.warning("A WARNING")
logger2.error("An ERROR")
logger2.critical("A message of CRITICAL severity")

=====logger1=====
INFO:logger1:An INFO
ERROR:logger1:An ERROR
CRITICAL:logger1:A message of CRITICAL severity
=====logger2=====
ERROR:logger2:An ERROR
CRITICAL:logger2:A message of CRITICAL severity


# Basic example

Here I create loggin system which will write any error to `logging_files/basic.log`

In [3]:
log_filename = "logging_files/basic.log"
fun_lines = [
    lambda: "no_bug", # no_bug function
    lambda: 7/0, # ZeroDivisionError
    lambda: "hello" + 5784 # TypeError
]

basic_logger = logging.getLogger("basic loger")
basic_logger.level = logging.INFO
handler = logging.FileHandler(log_filename, mode='w')
handler.setFormatter(
    logging.Formatter('%(asctime)s %(levelname)s %(message)s')
)
basic_logger.addHandler(handler)

for fun in fun_lines:
    try:
        fun()
    except:
        basic_logger.error("Any error", exc_info=True)

with open(log_filename) as file:
    print(file.read())

2023-06-30 08:58:14,800 ERROR Any error
Traceback (most recent call last):
  File "/tmp/ipykernel_7144/1911589912.py", line 18, in <module>
    fun()
  File "/tmp/ipykernel_7144/1911589912.py", line 4, in <lambda>
    lambda: 7/0, # ZeroDivisionError
ZeroDivisionError: division by zero
2023-06-30 08:58:14,800 ERROR Any error
Traceback (most recent call last):
  File "/tmp/ipykernel_7144/1911589912.py", line 18, in <module>
    fun()
  File "/tmp/ipykernel_7144/1911589912.py", line 5, in <lambda>
    lambda: "hello" + 5784 # TypeError
TypeError: can only concatenate str (not "int") to str



# Exception in exception

I wanted to learn how `looging` module will process the situation when there is one exception is handled within the framework of the other. In the following example there are functions:
- `inside_exception` calls the division by zero exception;
- `outside_exception` will execute `inside_exception` and in addition will lead to type exception;
- as a result we see only `TypeError` in log.

In [2]:
log_filename = "logging_files/excep_in_excep.log"
logging.basicConfig(
    level=logging.INFO, 
    filename=log_filename,
    filemode="w",
    format="%(asctime)s %(levelname)s %(message)s"
)

def inside_exception():
    try:
        7/0
    except:
        pass

def outside_exception():
    try:
        inside_exception()
        "hello" + 7
    except:
        logging.error("Any error", exc_info=True)

outside_exception()

with open(log_filename) as f:
    print(f.read())

2023-06-30 08:57:13,367 ERROR Any error
Traceback (most recent call last):
  File "/tmp/ipykernel_7144/3688385058.py", line 18, in outside_exception
    "hello" + 7
TypeError: can only concatenate str (not "int") to str



# Many logs to same file

Is it possible to store different logs in the same file? So in the following exmaple I create 20 loggers and bind them to the same file. Then I make 50 log messages each from random loggers. After that I output the resulting file. Everything is fine.

In [6]:
log_filename = "logging_files/manylog_samefile.log"
logs_count = 20
loggers = []

for i in range(logs_count):
    loger_name = f"logger {i}"
    basic_logger = logging.getLogger(loger_name)
    basic_logger.level = logging.INFO
    handler = logging.FileHandler(log_filename, mode='a')
    handler.setFormatter(
        logging.Formatter('%(message)s %(asctime)s %(levelname)s %(name)s')
    )
    basic_logger.addHandler(handler)
    loggers.append(basic_logger)

for i in range(0,50):
    loggers[random.randint(0, logs_count - 1)].error(f"Iteration{i}")

with open(log_filename) as f:
    print(f.read())
os.remove(log_filename)

Iteration0 2023-06-30 09:19:42,804 ERROR logger 9
Iteration1 2023-06-30 09:19:42,805 ERROR logger 9
Iteration2 2023-06-30 09:19:42,805 ERROR logger 9
Iteration3 2023-06-30 09:19:42,805 ERROR logger 19
Iteration4 2023-06-30 09:19:42,805 ERROR logger 19
Iteration5 2023-06-30 09:19:42,805 ERROR logger 17
Iteration6 2023-06-30 09:19:42,805 ERROR logger 6
Iteration7 2023-06-30 09:19:42,805 ERROR logger 0
Iteration8 2023-06-30 09:19:42,805 ERROR logger 1
Iteration9 2023-06-30 09:19:42,805 ERROR logger 8
Iteration10 2023-06-30 09:19:42,805 ERROR logger 7
Iteration11 2023-06-30 09:19:42,805 ERROR logger 8
Iteration12 2023-06-30 09:19:42,805 ERROR logger 10
Iteration13 2023-06-30 09:19:42,805 ERROR logger 8
Iteration14 2023-06-30 09:19:42,805 ERROR logger 8
Iteration15 2023-06-30 09:19:42,805 ERROR logger 17
Iteration16 2023-06-30 09:19:42,805 ERROR logger 12
Iteration17 2023-06-30 09:19:42,805 ERROR logger 3
Iteration18 2023-06-30 09:19:42,805 ERROR logger 13
Iteration19 2023-06-30 09:19:42,80

In this case it's important to use mode `mode='a'` for the file handler. The following cell is the same as the previous one, but uses `mode='w'` instead of `mode='a'`. Something is wrong with the logger in this case.

In [4]:
log_filename = "logging_files/manylog_samefile.log"
logs_count = 20
loggers = []

for i in range(logs_count):
    loger_name = f"logger {i}"
    basic_logger = logging.getLogger(loger_name)
    basic_logger.level = logging.INFO
    handler = logging.FileHandler(log_filename, mode='w')
    handler.setFormatter(
        logging.Formatter('%(message)s %(asctime)s %(levelname)s %(name)s')
    )
    basic_logger.addHandler(handler)
    loggers.append(basic_logger)

for i in range(0,50):
    loggers[random.randint(0, logs_count - 1)].error(f"Iteration{i}")

with open(log_filename) as f:
    print(f.read())
os.remove(log_filename)

Iteration40 2023-06-30 09:00:40,578 ERROR logger 18Iteration49 2023-06-30 09:00:40,579 ERROR logger 7
Iteration48 2023-06-30 09:00:40,579 ERROR logger 5
18
eration34 2023-06-30 09:00:40,578 ERROR logger 1ItIteration43 2023-06-30 09:00:40,578 ERROR logger 13
tion27 2023-06-30 09:00:40,578 ERROR logger 3
Iteration39 2023-06-30 09:00:40,578 ERROR logger 3
Iteration42 2023-06-30 09:00:40,578 ERROR logger 3

